# Overview
TensorFlow is a programming system in which you represent computations as graphs. Nodes in the graph are called ops (short for operations). An op(node) takes zero or more Tensors, performs some computation, and produces zero or more Tensors. A Tensor is a typed multi-dimensional array. For example, you can represent a mini-batch of images as a 4-D array of floating point numbers with dimensions [batch_size, height, width, channels].

A TensorFlow graph is a description of computations. To compute anything, a graph must be launched in a Session. A Session places the graph ops onto Devices, such as CPUs or GPUs, and provides methods to execute them. These methods return tensors produced by ops as numpy ndarray objects in Python, and as tensorflow::Tensor instances in C and C++.

In [1]:
# Start by importing the library
import tensorflow as tf

## The computation graph
TensorFlow programs are usually structured into a construction phase, that assembles a graph, and an execution phase that uses a session to execute ops in the graph.

For example, it is common to create a graph to represent and train a neural network in the construction phase, and then repeatedly execute a set of training ops in the graph in the execution phase.

TensorFlow can be used from C, C++, and Python programs. It is presently much easier to use the Python library to assemble graphs, as it provides a large set of helper functions not available in the C and C++ libraries.
### Building a graph
To build a graph start with ops that do not need any input (source ops), such as Constant, and pass their output to other ops that do computation.

The ops constructors in the Python library return objects that stand for the output of the constructed ops. You can pass these to other ops constructors to use as inputs.

The TensorFlow Python library has a default graph to which ops constructors add nodes. The default graph is sufficient for many applications. See the Graph class documentation for how to explicitly manage multiple graphs.

In [2]:
# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

The default graph now has three nodes: two constant() ops and one matmul() op. To actually multiply the matrices, and get the result of the multiplication, you must launch the graph in a session. This session execute outside of the python on the optimized C++ Tensorflow engine

## Launching the graph
After creating a graph, you can launch it. To launch a graph, create a Session object. Without arguments the session constructor launches the default graph.

The TensorFlow implementation translates the graph definition into executable operations distributed across available compute resources, such as the CPU or one of your computer's GPU cards. In general you do not have to specify CPUs or GPUs explicitly. TensorFlow uses your first GPU, if you have one, for as many operations as possible.

In [3]:
# Launch the default graph.
sess = tf.Session()

# To run the matmul op we call the session 'run()' method, passing 'product'
# which represents the output of the matmul op.  This indicates to the call
# that we want to get the output of the matmul op back.
#
# All inputs needed by the op are run automatically by the session.  They
# typically are run in parallel.
#
# The call 'run(product)' thus causes the execution of three ops in the
# graph: the two constants and matmul.
#
# The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)

# Close the Session when we're done.
sess.close()

[[ 12.]]


### Another way to lauch a session

In [4]:
with tf.Session() as sess:
  result = sess.run([product])
  print(result)  
  sess.close()

[array([[ 12.]], dtype=float32)]


If you have more than one GPU available on your machine, to use a GPU beyond the first you must assign ops to it explicitly. Use with...Device statements to specify which CPU or GPU to use for operations:

In [5]:
with tf.Session() as sess:
  with tf.device("/gpu:1"):
    matrix1 = tf.constant([[3., 3.]])
    matrix2 = tf.constant([[2.],[2.]])
    product = tf.matmul(matrix1, matrix2)
    print(result)
    sess.close()

[array([[ 12.]], dtype=float32)]


Devices are specified with strings. The currently supported devices are:

    "/cpu:0": The CPU of your machine.
    "/gpu:0": The GPU of your machine, if you have one.
    "/gpu:1": The second GPU of your machine, etc.


## Tensors
TensorFlow programs use a tensor data structure to represent all data -- only tensors are passed between operations in the computation graph. You can think of a TensorFlow tensor as an n-dimensional array or list. A tensor has a static type, a rank, and a shape.

### Variables

Variables maintain state across executions of the graph. Basically you can iterate on the graph and the variable values.
When implementing neural network models the variables are used to store weight and bias parameters

The following example shows a variable serving as a simple counter. See Variables for more details.

In [6]:
# Build a graph
# Create a Variable, that will be initialized to the scalar value 0.
varState = tf.Variable(0, name="counter")
one = tf.constant(1)
new_value = tf.add(varState, one)
# The assign() operation in this code is a part of the expression graph just like the add() operation, 
# so it does not actually perform the assignment until run() executes the expression.
update = tf.assign(varState, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'varState'
  print(sess.run(varState))
  # Run the op that updates 'varState' and print 'varState' content.
  for cnt in range(3):
    print('Counter=%d' % cnt)
    # Iterate graph
    sess.run(update)
    # Get variable result
    print('Variable value is %d' % sess.run(varState))
  sess.close()


0
Counter=0
Variable value is 1
Counter=1
Variable value is 2
Counter=2
Variable value is 3


### Fetches

To fetch the outputs of operations, execute the graph with a run() call on the Session object and pass in the tensors to retrieve. In the previous example we fetched the single node state, but you can also fetch multiple tensors:

In [7]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:  
  result = sess.run([mul, intermed, input3, input2, input1])
  print(result)
  sess.close()

[21.0, 7.0, 5.0, 2.0, 3.0]


### Feeds
Basically it will alow you to enter custom information (ex images) on your graph, before you run it.

The examples above introduce tensors into the computation graph by storing them in Constants and Variables. TensorFlow also provides a feed mechanism for patching a tensor directly into any operation in the graph.

A placeholder() operation generates an error if you do not supply a feed for it.

A feed temporarily replaces the output of an operation with a tensor value. You supply feed data as an argument to a run() call. The feed is only used for the run call to which it is passed. The most common use case involves designating specific operations to be "feed" operations by using tf.placeholder() to create them:

In [5]:
# Simple multiplication graph
in_1 = tf.placeholder(tf.float32)
in_2 = tf.placeholder(tf.float32)
inter = tf.mul(in_1, in_2)
output = tf.add(inter, tf.constant(1.0))

# Open session
init = tf.initialize_all_variables()
sess = tf.Session()
writer = tf.train.SummaryWriter("./logs/multSimple", sess.graph)

# Basically we can add values to the feeds before execution
result = sess.run([output], feed_dict={in_1:[20.], in_2:[3.]})

# Close the Session when we're done.
sess.close()
print(result)

[array([ 61.], dtype=float32)]


## Tensorboard
Is a tool on tensorflow that allows you to vizualize graphs and information running on your session
<img src="SimpleMult.png">

The above code save the graph on tensorboard
tensorboard --logdir=logs
http://localhost:6006/#graphs